In [ ]:
# a list of all goals! 
# Home and Away and scoring team
# time of goals
# in or out of box

In [ ]:
from floodlight.io import dfl
from floodlight.io import opta
import numpy as np
import pandas as pd
import fileinput
import os
import csv
import glob
from lxml import etree
import pickle

event_types = pd.read_excel('OtherFiles/Opta_Event_Types_Cleaned.xlsx', index_col=0)
event_types[5:20] # 16 = goals

with open('Output/game_names_DFL.pkl', 'rb') as f:
    game_names_DFL = pickle.load(f)

with open('Output/game_names_Opta.pkl', 'rb') as f:
    game_names_Opta = pickle.load(f)
    

with open('Output/game_MD_Opta.pkl', 'rb') as f:
    game_MD_Opta = pickle.load(f)
    


# 16	Small box-centre
# 17	Box-centre
# 18	Out of box-centre
# 19	35+ centre
# 60	Small box-right
# 61	Small box-left
# 62	Box-deep right
# 63	Box-right
# 64	Box-left
# 65	Box-deep left
# 66	Out of box-deep right
# 67	Out of box-right
# 68	Out of box-left
# 69	Out of box-deep left
# 70	35+ right
# 71	35+ left

# 9	Penalty
# 5	Free kick taken
# 6	Corner taken
# 22	Regular play
# 23	Fast break
# 24	Set piece
# 25	From corner
# 26	Free kick
# 28	own goal



In [ ]:
opta_matchname = []
Home_Team = []
Away_Team = []
Scoring_Team = []
Minute = []
Second = []
InBox = []
HalfTime = []
OpenPlay = []



ROOT = os.getcwd() # returns current working directory of a process.

ev_data = (r"EventData") # creates a raw string which treats backslash (\) as a literal character. 

for dir in os.listdir(ev_data):
    if dir.startswith('Spieltag'):
        md = dir
        ST = md.replace('Spieltag ', '')# get the numerical matchday
        loc = os.path.join(ev_data, dir) 
        # print(loc)
        for eve_file in os.listdir(loc):
            # print(eve_file)
            if eve_file.startswith('f'):
                opta_mn = eve_file.replace('-eventdetails.xml', '')
                tree = etree.parse(str('/'.join([loc, eve_file])))
                root = tree.getroot()
                matchinfo = root.xpath("Game")[0].attrib

                
                # get goal location
                Events = opta.read_event_data_xml(str('/'.join([loc, eve_file])))[0]      # read the events
                halfs = ['HT1', 'HT2']
                teams = ['Home', 'Away']
                
                for h in halfs:
                    half = 1
                    half = 2 if h == 'HT2' else half
                    for t in teams:
                        scoring_team = matchinfo['home_team_name']
                        scoring_team = matchinfo['away_team_name'] if t == 'Away' else scoring_team  # which teams is the scoring team
                        data = Events[h][t].events
                        goals = data[data.eID == 16] 
                        for g in range(len(goals)):   # loop over goals in that half for that team
                            opta_matchname.append(opta_mn)
                            Home_Team.append(matchinfo['home_team_name'])
                            Away_Team.append(matchinfo['away_team_name'])
                            Scoring_Team.append(scoring_team)
                            HalfTime.append(half)
                            Minute.append(goals.minute.iloc[g])
                            Second.append(goals.second.iloc[g])
                            # check the goal location in qualifier
                            if any(x in goals.iloc[g].qualifier for x in ('16: None', '17: None', '60: None', '61: None', '62: None', '63: None', '64: None', '65: None')):
                                inbox = True
                            elif any(x in goals.iloc[g].qualifier for x in ('18: None', '19: None', '66: None', '67: None', '68: None', '69: None', '70: None', '71: None')):
                                inbox = False
                            else:
                                inbox = 'Undeterminable'
                            InBox.append(inbox)
                            
                            # check if from open play
                            if any(x in goals.iloc[g].qualifier for x in ('22: None', '23: None')):
                                op = True
                            elif any(x in goals.iloc[g].qualifier for x in ('24: None', '25: None')):
                                op = 'Indirect'
                            elif any(x in goals.iloc[g].qualifier for x in ('9: None', '26: None')):
                                op = 'direct'
                            else:
                                op = 'Undeterminable'
                                
                            OpenPlay.append(op)
                                
                                
goal_overview = pd.DataFrame(list(zip(opta_matchname, Home_Team, Away_Team, Scoring_Team, HalfTime, Minute, Second, InBox, OpenPlay)),
                             columns=['Opta_Name', 'Home', 'Away', 'Scoring_Team', 'Half', 'Minute', 'Second', 'InBox', 'OpenPlay'])


goal_overview 

In [ ]:
GO = pd.read_csv('Output/GameOverview3.csv')
GO

In [ ]:
goal_overview['DFL_Name'] = goal_overview.Opta_Name
goal_overview['Matchday'] = goal_overview.Opta_Name


type(list(game_names_Opta.keys()))
goal_overview = goal_overview[goal_overview['Opta_Name'].isin(list(game_names_Opta.keys()))]

goal_overview['DFL_Name'] = goal_overview['DFL_Name'].apply(lambda x: game_names_Opta[x])
goal_overview['Matchday'] = goal_overview['Matchday'].apply(lambda x: game_MD_Opta[x])

goal_overview.to_csv('Output/GoalOverview.csv')

goal_overview


In [ ]:
goal_overview.InBox.value_counts()

In [ ]:
goal_overview.OpenPlay.value_counts()

In [3]:
import pandas as pd

GO = pd.read_csv('Output/GoalOverview.csv')
GO.InBox.value_counts()


True     760
False    104
Name: InBox, dtype: int64

In [2]:
GO.OpenPlay.value_counts()

True        605
Indirect    161
direct       98
Name: OpenPlay, dtype: int64

In [4]:
GO.groupby(["OpenPlay", "InBox"]).size()

OpenPlay  InBox
Indirect  False      5
          True     156
True      False     77
          True     528
direct    False     22
          True      76
dtype: int64